In [ ]:
!pip install pymysql sqlalchemy pandas

In [ ]:
# Length of general bin: 49cm - we take it as 46cm cause must minus the length of the setup
# Length of sanitary bin: 54cm - we take it as 51cm
# Treshhold of both bins: 75%
# Distance of toilet paper from sensor when full - 2cm
# Distance of toilet paper from sensor when empty - 8.5cm

In [ ]:
# Import Statements
import paho.mqtt.client as mqtt
import logging
import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import requests



normalBinMax = 46
sanBinMax = 65
toiletpaperMax = 8.5

# Enviroment Settings
telegramURL = "https://api.telegram.org/bot733055974:AAEsyKoq3z5mquv0k_xmzugu_dej_1MbdtA/sendMessage?chat_id=-1001267215209&parse_mode=Markdown&text="
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S',  level=logging.DEBUG)
logger = logging.getLogger(__name__)
MQTT_BROKER_HOSTNAME="broker.mqttdashboard.com"
database_settings = { "username": "admin",
                     "password": "abhyuday98",
                     "ip_address":"iotg5.ccspnys1uzca.us-east-1.rds.amazonaws.com",
                     "port": "3306",
                     "database_name": "iotg5"
                            }
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(database_settings['username'],
                                                                              database_settings['password'],
                                                                              database_settings['ip_address'],
                                                                              database_settings['port'],
                                                                              database_settings['database_name']
                                                                              ))
lastNdataPackets = 3
thresholds = {
    '1' : 8,
    '2' : 8,
    '3' : 11,
    '4' : 10

}
rangeThres = 1
lastValue = {
    '1':0,
    '2':0,
    '3':0,
    '4':0
}
lastNdatPacketsPerTable = {
    '1' : [],
    '2' : [],
    '3' : [],
    '4' : []

}
tableNames = {
    '1' : 'toiletpaper1',
    '2' : 'toiletpaper2',
    '3' : 'litterbin1',
    '4' : 'litterbin2'
}
telegramMessage = {
    '1': 'Female toilet: \nCubicle 4\nToilet paper dispenser is empty',
    '2': 'Female toilet: \nCubicle 4\nToilet paper dispenser is empty',
    '3': 'Female toilet: \nTrash bin is full',
    '4': 'Female toilet: \nCubicle 4\nSanitary bin is full',
}

In [ ]:

def demo_b2g_on_connect(client, userdata, flags, rc):

    # topic to subscribe to.
    # {getpass.getuser()} and {random.randint(1, 100)} are the username and a random number on your PC, independent of the MQTT client/broker
    topic = f"iotg5/podc"
    print(f"Connected to {MQTT_BROKER_HOSTNAME}. Result code: {rc}")

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(topic)
    print(f"Subscribed to topic: {topic}")
    print(f"Publish something to {topic} and the messages will appear here.")


# Callback for when a message is received from the MQTT broker.
def demo_b2g_on_message(client, userdata, msg):
    try:
        topic = "iotg5/cp"

        print(f"topic: {msg.topic}  Message: {msg.payload.decode('utf8')}")
        params = msg.payload.decode('utf8').split('?')[1].split('&')
        for param in params:
            split_param = param.split('=')
            index = split_param[0]
            dist = split_param[1]
            if index in tableNames:
                checkThreshold(index, dist)
    except Exception as e:
        print(e)
# Broker 🠒 gateway

def updateDatabase(sensorId, dist):
    prevValue = float(lastValue[sensorId])
    condition = float(dist)!=prevValue
#     if sensorId=='3' or sensorId=='4':
#         condition = float(dist)!=prevValue and float(dist)!=(prevValue-rangeThres) and float(dist)!=(prevValue+rangeThres)
        
#     print(lastValue[sensorId]!=dist, (lastValue[sensorId]+rangeThres)!=dist, (lastValue[sensorId]-rangeThres)!=dist)
    if condition:
        with engine.connect() as con:
            statement = text("INSERT INTO {} (`SensorId`, `DateTime`, `Distance(cm)`, `createdAt`, `updatedAt`) VALUES ('{}', CONVERT_TZ(now(),'+00:00','+08:00') ,'{}', CONVERT_TZ(now(),'+00:00','+08:00'), CONVERT_TZ(now(),'+00:00','+08:00'))".format(tableNames[sensorId], sensorId, dist))
            con.execute(statement)
            print('sensor', sensorId, 'with distance',dist, 'interted into ', tableNames[sensorId],'\n addedRecord')
    else: 
        try:
            with engine.connect() as con:
                statement = text("select `tableId` from {} order by DateTime desc limit 1".format(tableNames[sensorId]))
                result = con.execute(statement)
                tableId = 0
                for row in result:
                    tableId = str(row['tableId'])
                statement = text("update {} set updatedAt=CONVERT_TZ(now(),'+00:00','+08:00'), `DateTime`=CONVERT_TZ(now(),'+00:00','+08:00')  where `tableId`={}".format(tableNames[sensorId], tableId))
                con.execute(statement)
                print('sensor', sensorId, 'with distance',dist, 'interted into ', tableNames[sensorId],'\n updatedRecord')
        except:
            print('E')
    lastValue[sensorId]=dist
def updateLastNThreshold(sensorId):
    resultList = []
    with engine.connect() as con:
        statement = text("select `Distance(cm)` from {} order by DateTime desc limit {}".format(tableNames[sensorId],lastNdataPackets))
        result = con.execute(statement)
        for row in result:
            resultList.append(float(row['Distance(cm)']))
    #         print("row:", row['Distance(cm)'])
    return resultList

        
def checkThreshold(sensorId, dist):
    updateDatabase(sensorId, dist)
#     if sensorId == '1' or sensorId == '2':
#         checkDoubleState('1','2')
#     else:
#         CheckSingleState(sensorId)
                
def CheckSingleState(sensorId):
    resultListLastNThreshold = updateLastNThreshold(sensorId)
    print(resultListLastNThreshold)
    if(len(resultListLastNThreshold)==lastNdataPackets):
        flag = True
        for value in resultListLastNThreshold:
            thresCondition = value>thresholds[sensorId]
            print('sensorId: ',sensorId )
            print('value: ',value )
            print('thresholds: ',thresholds[sensorId])
            if thresCondition:
                flag = False
                break
        if flag:
            sendTelegramMessage(telegramMessage[sensorId]) 
        else:
            sendEmptyStatus(sensorId)

                
def checkDoubleState(sensorId1, sensorId2):
    paperRoll1 = updateLastNThreshold(sensorId1)
    paperRoll2 = updateLastNThreshold(sensorId2)
    flagPaperRoll1 = True
    flagPaperRoll2 = True
    if(len(paperRoll1)==lastNdataPackets):
        for value in paperRoll1:
            thresCondition = value<thresholds[sensorId1]
            print('sensorId: ',sensorId1)
            print('value: ',value )
            print('thresholds: ',thresholds[sensorId1])
            if thresCondition:
                flagPaperRoll1 = False
                break
    if(len(paperRoll2)==lastNdataPackets):
        for value in paperRoll2:
            thresCondition = value<thresholds[sensorId2]
            print('sensorId: ',sensorId2)
            print('value: ',value )
            print('thresholds: ',thresholds[sensorId2])
            if thresCondition:
                flagPaperRoll2 = False
                break
    if flagPaperRoll1 and flagPaperRoll2:
        sendFullStatus(sensorId1)

    else:
        sendEmptyStatus(sensorId1)
    

def sendFullStatus(sensorId):
    sendTelegramMessage(telegramMessage[sensorId]) 

def sendEmptyStatus(sensorId):
    print('Refilled', sensorId)
def sendTelegramMessage(bot_message):
    response = requests.get(telegramURL + bot_message)
    response = response.json()
    


def demo_b2g():
    # create mqtt client and assign callback functions
    client = mqtt.Client()
    client.on_connect = demo_b2g_on_connect
    client.on_message = demo_b2g_on_message

    # connect to broker
    client.connect(MQTT_BROKER_HOSTNAME, 1883, 60)

    # loop infinitely
    while True:
        # wait for timeout=1 seconds, process any events during that 1s, then return. See https://pypi.org/project/paho-mqtt/#network-loop
        client.loop(timeout=1)

if __name__ == '__main__': 
    demo_b2g()

Connected to broker.mqttdashboard.com. Result code: 0
Subscribed to topic: iotg5/podc
Publish something to iotg5/podc and the messages will appear here.
topic: iotg5/podc  Message: https://e.io/?1=10&2=9
sensor 1 with distance 10 interted into  toiletpaper1 
 addedRecord
sensor 2 with distance 9 interted into  toiletpaper2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=9
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 9 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Me

topic: iotg5/podc  Message: https://e.io/?1=10&2=8
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 8 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=8
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 8 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=8
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 8 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=8
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 8 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=8
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor

topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=9
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 9 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=9
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 9 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 

topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=2
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 2 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=2
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 2 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 

topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 wi

...

topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=2
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 2 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=2
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 2 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 

topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?4=0
sensor 4 with distance 0 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=1
sensor 4 with distance 1 interted into  litterbin2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=1
sensor 4 with distance 1 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=2
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with d

topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=82
sensor 4 with distance 82 interted into  litterbin2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=78
sensor 4 with distance 78 interted into  litterbin2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=78
sensor 4 with distance 78 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc

topic: iotg5/podc  Message: https://e.io/?4=69
sensor 4 with distance 69 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=30
sensor 4 with distance 30 interted into  litterbin2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=48
sensor 4 with distance 48 interted into  litterbin2 
 addedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=14
sensor 4 with distance 14 interted into  litterbin2 
 addedRecord
topic: iotg5/podc

topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted into  toiletpaper1 
 updatedRecord
sensor 2 with distance 3 interted into  toiletpaper2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?3=5
sensor 3 with distance 5 interted into  litterbin1 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=61
sensor 4 with distance 61 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?4=61
sensor 4 with distance 61 interted into  litterbin2 
 updatedRecord
topic: iotg5/podc  Message: https://e.io/?1=10&2=3
sensor 1 with distance 10 interted i

In [ ]:
params = url.split('?')[1].split('&')
    for param in params:
        split_param = param.split('=')
        index = split_param[0]
        dist = split_param[1]
        if(int(dist) == 0):
            ubit.pixels = [0b00000,
                           0b00000,
                           0b00000,
                           0b00000,
                           0b00000]
        if(int(dist) == 1):
            ubit.pixels = [0b00100,
                           0b00100,
                           0b00100,
                           0b00100,
                           0b00100]
        if(int(dist) == 2):
            ubit.pixels = [0b01010,
                           0b01010,
                           0b01010,
                           0b01010,
                           0b01010]
        if(int(dist) == 3):
            ubit.pixels = [0b10101,
                           0b10101,
                           0b10101,
                           0b10101,
                           0b10101]

In [ ]:
'''
with engine.connect() as con:
            if index == '1':
                statement = text("INSERT INTO toiletpaper1 (`SensorId`, `Distance(cm)`) VALUES ('1', '{}')".format(dist))
                con.execute(statement)
                toiletpaper1.append(float(dist))
                print('sensor', 1, 'with distance',dist, 'interted into toiletpaper1')
            if index == '2':
                statement = text("INSERT INTO toiletpaper2 (`SensorId`, `Distance(cm)`) VALUES ('2', '{}')".format(dist))
                con.execute(statement)
                toiletpaper2.append(float(dist))

                print('sensor', 2, 'with distance',dist, 'interted into toiletpaper2')

                
            if index == '3':
                statement = text("INSERT INTO litterbin1 (`SensorId`, `Distance(cm)`) VALUES ('3', '{}')".format(dist))
                con.execute(statement)
                litterbin1.append(float(dist))

#                 print('sensor', 3, 'with distance',dist, 'interted into litterbin1')
                
            if index == '4':
                statement = text("INSERT INTO litterbin2 (`SensorId`, `Distance(cm)`) VALUES ('4', '{}')".format(dist))
                con.execute(statement)
                litterbin2.append(float(dist))

#                 print('sensor', 4, 'with distance',dist, 'interted into litterbin2')

            if float(dist)>=9:
                print("Dispenser is Empty!")
            
            if(len(toiletpaper1)==lastNdataPackets){
                total = 0
                for value in toiletpaper1:
                    total += value
                total = total//lastNdataPackets
                if total == toiletpaper1[lastNdataPackets-1]:  
            }
'''

In [ ]:
with engine.connect() as con:
    statement = text("select `Distance(cm)` from {} order by DateTime desc limit {}".format('litterbin2',lastNdataPackets))
    result = con.execute(statement) 
    for row in result:
        print("row:", float(row['Distance(cm)']))
